# Importações Iniciais e criação de descritor para HOG


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import data, exposure

from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
#window, block, blockStride,cell, bins 
#hog = cv2.HOGDescriptor((25, 25), (14, 14), (7, 7), (14, 14), 8)
hog = cv2.HOGDescriptor((32, 32), (8, 8), (4, 4), (8, 8), 8) 
#a janela tem um tamanho diferente do artigo pois 25x25 não funciona com o método do opencv,
#já que não é multiplo dos outros tamanhos

#usei o mais proximo acima

# Pega dados de um arquivo de imagem e cria array numpy para guardar os resultados

In [2]:
import glob
size = 200, 200
hasPothole = []

i = 0;

potholeFolderSize = len(glob.glob('potholes/*.jpg'))

firstFileName = ""
for filename in glob.glob('potholes/*.jpg'): 
    firstFileName = filename
    break

    
im=cv2.imread(firstFileName)

im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogImage = hog.compute(image_resized).reshape(1, -1)


potHoleImageDimensions =hogImage.shape[1]         
    
potholeData = np.zeros((potholeFolderSize, potHoleImageDimensions))

    

# Calcula HOG para todas as imagens de buracos na pasta. Segue o seguinte processo: Lê imagem, converte para tons de cinza, redimensiona para 200x200 e por fim calcula HOG

In [3]:
i = 0;
for filename in glob.glob('potholes/*.jpg'): 
   
    
    im=cv2.imread(filename)
    #print(filename)
    im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

   
    image_resized = cv2.resize(im_r, (size))
    hasPothole.append(image_resized)
    potholeData[i] = (hog.compute(image_resized).reshape(1, -1))
    i= i + 1

# Mesmo processo acima mas para imagens sem buracos

In [4]:
normalRoadsFolderSize = len(glob.glob('normal/*.jpg'))

for filename in glob.glob('normal/*.jpg'): 
    firstFileName = filename
    break


im=cv2.imread(firstFileName)

im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogImage = hog.compute(image_resized).reshape(1, -1)


normalRoadsmageDimensions =hogImage.shape[1]         
    
normalRoadsData = np.zeros((normalRoadsFolderSize, normalRoadsmageDimensions))

In [5]:

i = 0;
for filename in glob.glob('normal/*.jpg'): 
   
    #print(filename)
    im=cv2.imread(filename)
   # im = np.asarray(im, dtype=np.uint8)
    
    im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

   
    image_resized = cv2.resize(im_r, (size))
    #hasPothole.append(image_resized)
    normalRoadsData[i] = (hog.compute(image_resized).reshape(1, -1))
    i= i + 1

# Cria dados para treinamento e efetua teste com GaussianNB. Faz Predição de imagem sem buraco (0), e imagem com buraco (1)

In [6]:
from itertools import chain
X_train = np.array([hogData
                    for hogData in chain(potholeData,
                                    normalRoadsData)])
y_train = np.zeros(X_train.shape[0])
y_train[:potholeData.shape[0]] = 1

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

gnb = GaussianNB()
gnb.fit(X_train, y_train)

#sem buraco
im=cv2.imread("normal/115.jpg")
im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogP = (hog.compute(image_resized).reshape(1, -1))

print(gnb.predict(hogP))

#com buraco
#sem buraco
im=cv2.imread("potholes/115.jpg")
im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogP = (hog.compute(image_resized).reshape(1, -1))
print(gnb.predict(hogP))

[0.]
[1.]


# Implementando teste com SVM dado em exemplo https://jakevdp.github.io/PythonDataScienceHandbook/05.14-image-features.html

# Só para completude. Não faz parte da implementação do artigo. Resultados são os mesmos basicamente. Este processo demora

# Efetua Grid Search e fit

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection  import GridSearchCV

grid = GridSearchCV(LinearSVC(dual=False,max_iter=1000), {'C': [1.0, 10.0, 100.0, 1000.0]})
grid.fit(X_train, y_train)

In [ ]:
model = grid.best_estimator_
model.fit(X_train, y_train)

# Testa classificação. 1 é buraco, 0 é ausência.

In [ ]:
im=cv2.imread("potholes/115.jpg")
im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogP = (hog.compute(image_resized).reshape(1, -1))


In [ ]:
labels = model.predict(hogP)
labels.sum()

In [ ]:
im=cv2.imread("normal/1.jpg")
im_r= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
image_resized = cv2.resize(im_r, (size))
hogP = (hog.compute(image_resized).reshape(1, -1))

labels = model.predict(hogP)
labels.sum()

# Fazendo segmentação da imagem com buraco para mostrar área... código retirado de https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_rag_merge.html#sphx-glr-auto-examples-segmentation-plot-rag-merge-py

# Não é o usado por fim no artigo mas deu um resultado relativamente similar ao apresentado no artigo, para imagens vistas de cima.

In [ ]:
from skimage import data, io, segmentation, color
from skimage.future import graph
import numpy as np


def _weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.

    The method expects that the mean color of `dst` is already computed.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.

    This method computes the mean color of `dst`.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                      graph.nodes[dst]['pixel count'])


img = cv2.imread("potholes/115.jpg")
labels = segmentation.slic(img, compactness=30, n_segments=400)
g = graph.rag_mean_color(img, labels)

labels2 = graph.merge_hierarchical(labels, g, thresh=40, rag_copy=False,
                                   in_place_merge=True,
                                   merge_func=merge_mean_color,
                                   weight_func=_weight_mean_color)

out = color.label2rgb(labels2, img, kind='avg', bg_label=0)
out = segmentation.mark_boundaries(out, labels2, (0, 0, 0))
#io.imshow(out)
#io.show()

fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))

ax[0].imshow(out)
ax[1].imshow(img)

for a in ax:
    a.axis('off')

plt.tight_layout()